In [1]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import sys
sys.tracebacklimit = 0

import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt

: 

In [ ]:
from pettingzoo.test import parallel_api_test
from solution.custom_gym import CustomGymEnviornment
from solution.trainer import *
from solution.policy_net import *

In [ ]:
from core.agent import *
from core.world import * 
from core.render import * 
from core.skill import * 
from core.models import *
from core.message import *

In [ ]:
from sar.sar_agent import *
from sar.sar_world import *
from sar.sar_env_params import *

belief_initializer = SARBeliefInitializer(BELIEF_DIMS)
def initialize_swarm(world : BaseWorld):
    swarm = [SARAgent() for i in range(SWARM_SIZE)]
    for agent in swarm:
        agent.set_utility(SARUtilityFunction())
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    swarm = belief_initializer.initialize_beliefs(swarm)

In [ ]:

from sar.urban_gen import * 
from sar.victims import * 
from sar.sar_comm import * 

terrain_generator = UrbanTerrainMapGenerator(padding = MAX_VISIBILITY)
victim_generator = VictimGenerator(padding = MAX_VISIBILITY)
def initialize_terrain(world : BaseWorld):
    terrain_map, population_map = terrain_generator.generate(world._dims)
    map_collection : BaseMapCollection = BaseMapCollection()
    map_collection.add_map("Terrain", terrain_map)
    map_collection.add_map("Population", population_map)

    victim_generator.set_density_map(population_map)
    victim_map = victim_generator.generate(world._dims)

    map_collection.add_map("Victims", victim_map)
    return map_collection


In [ ]:
from sar.energy import EnergyModel
from sar.victims import VictimModel
from solution.sar_action_interpreter import *
from solution.encoder_net import *

world = SARWorld(dims = WORLD_DIMS,
              swarm_initializer= initialize_swarm,
              generation_pipeline=initialize_terrain
              )
world.add_model("energy_model", EnergyModel())
world.add_model("victim_model", VictimModel())
world.reset()

In [ ]:
# TODO: Potentially refactor encoder_model, decoder_model and policy_net to be in one class for convenience.
encoder_model = Encoder()
decoder_model = Decoder()
comms_protocol = SARCommunicationProtocol(encoder_model, decoder_model)
action_interpreter = SARActionInterpreter(BELIEF_DIMS)

custom_gym : CustomGymEnviornment = CustomGymEnviornment(world, action_interpreter, comms_protocol)

In [ ]:
custom_gym.reset(42)

### Testing

In [ ]:
parallel_api_test(custom_gym, num_cycles=1_000)
custom_gym.reset()

# Training

In [ ]:

from models.base import * 
from models.idqn import * 
from solution.policy_net import PolicyNet

In [ ]:
model = IDQN(env = custom_gym,
             feature_extractor= feature_extractor,
             policy_net= PolicyNet(1, 7, 4), 
             target_net=  PolicyNet(1, 7, 4),
             encoder_net = encoder_model,
             decoder_net= decoder_model,
             batch_size=1024
             )

In [ ]:
train_loop(custom_gym, model, games=10, seed=42)